In [ ]:
docker run --rm -v C:\Users\ferli\Documents\ShroomRadar:/data ingmapping/tippecanoe tippecanoe -f -e /data/tiles -Z6 -z14 --no-tile-compression --no-feature-limit --no-tile-size-limit --preserve-input-order /data/shroomradar/data/outputs/grid_tuscany_with_topography_predictions.geojson


In [ ]:


import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.cloud import storage

# Local tiles directory
tiles_dir = r"C:\Users\ferli\Documents\ShroomRadar\tiles"

# GCS client + bucket
client = storage.Client()
bucket = client.bucket("mushroom-radar-tiles")

# Collect all files first
file_list = []
for root, _, files in os.walk(tiles_dir):
    for file in files:
        local_path = os.path.join(root, file)
        remote_path = "tiles/" + os.path.relpath(local_path, tiles_dir).replace("\\", "/")
        file_list.append((local_path, remote_path))

print(f"Found {len(file_list)} files to upload")

def upload_file(local_remote):
    local_path, remote_path = local_remote
    blob = bucket.blob(remote_path)
    blob.cache_control = "no-store"  # avoid CDN caching issues
    blob.upload_from_filename(local_path)
    return remote_path

# Upload in parallel
max_workers = 32  # tune based on CPU/connection
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(upload_file, f): f for f in file_list}
    for i, future in enumerate(as_completed(futures), 1):
        try:
            remote_path = future.result()
            if i % 1000 == 0:  # print every N uploads
                print(f"[{i}/{len(file_list)}] Uploaded {remote_path}")
        except Exception as e:
            print(f"❌ Error uploading {futures[future][1]}: {e}")


Found 9780 files to upload
